# =============================================
## GHI CHÚ TỔNG QUAN (NOTE – DÀNH CHO KAGGLE)
# =============================================
 **Mục tiêu**
 - Fine-tune mô hình T5 (seq2seq) cho bài toán Text-to-SQL trên Kaggle.
 - Dùng dataset công khai (b-mc2/sql-create-context, Clinton/Text-to-sql-v1, knowrohit07/know_sql),
 sau đó trộn, tiền xử lý (tokenize), huấn luyện, lưu mô hình.
 - Không thay đổi logic cốt lõi; chỉ bổ sung chú thích chi tiết.

 **Luồng chính**
 1) Cài đặt thư viện → Import.
 2) Nạp/ghép dataset có sẵn → lưu cache (merged_dataset).
 3) Tiền xử lý (tokenize) → lưu cache (tokenized_datasets).
 4) Kiểm thử zero-shot (baseline) trên model gốc.
 5) Huấn luyện full fine-tune → lưu mô hình đã fine-tune.
 6) Kiểm thử lại với mô hình fine-tuned + đánh giá (ROUGE) mẫu nhỏ.
 7) Đóng gói mô hình (zip) để tải xuống / dùng về sau.

 **Đầu ra (sản phẩm)**
 - Thư mục model đã fine-tune: /kaggle/working/sql_t5_finetuned
 - Có thể nạp về app (FastAPI/Flask/.NET-bridge) bằng from_pretrained("/kaggle/working/sql_t5_finetuned").

In [ ]:
# Cài đặt thư viện datasets nếu chưa có
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 40.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.

In [ ]:
# Cài đặt thư viện evaluate
!pip install evaluate 


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


# ==============================
# 2) IMPORT CÁC THƯ VIỆN CẦN THIẾT
# ==============================

In [ ]:
import torch; 
print(torch.__version__, torch.cuda.is_available())

2.6.0+cu124 True


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, load_from_disk # Dataset Huggingface
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer # Model và Tokenizer cho trainning
import torch # Thư viện PyTorch tăng tốc độ xử lý GPU, tensor
import time # Ghi thời gian trainning
import evaluate # Thư viện evaluate để đánh giá mô hình (BLEU, ROUGE)
import pandas as pd # Thư viện pandas để xử lý dữ liệu dạng bảng/kết quả
import numpy as np # Thư viện numpy để xử lý mảng


import warnings
warnings.filterwarnings("ignore") # Bỏ qua các cảnh báo không cần thiết

2025-11-02 10:33:45.611153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762079625.772075      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762079625.822072      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Import thư viện transformers và huggingface_hub
!pip install transformers huggingface_hub 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 10.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration # Model và Tokenizer T5

import os # Thư viện os để thao tác với hệ thống tệp

# ==============================
# 3) CẤU HÌNH ĐƯỜNG DẪN VÀ THIẾT BỊ
# ==============================

In [ ]:
drive_model_path = '/kaggle/working/' # Đường dẫn lưu model trên Kaggle

In [ ]:
torch.cuda.is_available() # Kiểm tra GPU có sẵn không, trả về True hoặc False

True

In [ ]:
from huggingface_hub import snapshot_download

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Tải snapshot repo model về LOCAL FOLDER (chỉ gọi API 1 lần, không đụng 'additional_chat_templates')
#   - repo_id 't5-small' sẽ redirect về 'google-t5/t5-small'
local_dir = snapshot_download(repo_id="t5-small") 

Fetching 20 files:   0%|          | 0/20 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/8.47k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

onnx/decoder_model_merged_quantized.onnx:   0%|          | 0.00/58.7M [00:00<?, ?B/s]

onnx/decoder_model_quantized.onnx:   0%|          | 0.00/58.4M [00:00<?, ?B/s]

...

rust_model.ot:   0%|          | 0.00/242M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/242M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

In [ ]:
model_name='t5-small' # Tên model nền; code gốc chọn t5-small cho baseline/finetune

# tokenizer = T5Tokenizer.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(local_dir, local_files_only=True)

# Trên Kaggle GPU có thể dùng bfloat16, còn local CPU thì để mặc định FP32
if device == "cuda":
    original_model = T5ForConditionalGeneration.from_pretrained(
        model_name, torch_dtype=torch.bfloat16
    ).to(device)
else:
    original_model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

import warnings
warnings.filterwarnings("ignore", message="You are using the default legacy behaviour")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


# ==============================
# 4 ) NẠP/CHUẨN BỊ DATASET (CÓ CACHE)
# ==============================

# ==== Load OLSM from RAW JSON/JSONL và clean data ====

In [ ]:
# ==== ONE-CELL: Load OLSM from RAW JSON/JSONL (tự sửa & split) ====
from datasets import load_dataset
import json, re, os, codecs

RAW_FILES = [
    "/kaggle/input/olsm-data/dataset-olsm.json",
    "/kaggle/input/olsm-data/dataset-olsm2.json",
    "/kaggle/input/dataset-basic/dataset-basic.json",
    "/kaggle/input/dataset-basic/dataset-noizy.json",
]
OUT = "/kaggle/working/olsm-clean.jsonl"
os.makedirs(os.path.dirname(OUT), exist_ok=True)

# thay “smart quotes” -> ASCII
SMART = str.maketrans({
    "“": '"', "”": '"', "„": '"', "«": '"', "»": '"',
    "‘": "'", "’": "'", "‚": "'", "‹": "'", "›": "'",
})

def _fix_inner_quotes(text: str, field: str) -> str:
    # Đổi dấu " bên trong giá trị field -> ' và escape \n,\t
    pat = re.compile(rf'"{field}"\s*:\s*"(.*?)"', re.DOTALL)
    def repl(m):
        v = m.group(1)
        v = v.replace('\\"', '"')        # bỏ escape cũ lộn xộn
        v = v.replace('"', "'")          # " -> '
        v = v.replace("\r\n","\n").replace("\r","\n")
        v = v.replace("\n","\\n").replace("\t","\\t")
        return f'"{field}":"{v}"'
    return pat.sub(repl, text)

def _objects_from_text(txt: str):
    # nếu là JSON array chuẩn -> parse một lần
    t = txt.strip()
    if t.startswith("[") and t.endswith("]"):
        try:
            arr = json.loads(t)
            for obj in arr:
                yield obj
            return
        except Exception:
            pass
    # còn lại coi như JSONL "lẫn lộn": tách các object dính liền }{
    txt = re.sub(r"}\s*{", "}\n{", txt)
    for line in txt.splitlines():
        L = line.strip().rstrip(",")
        if not L:
            continue
        try:
            yield json.loads(L)
        except Exception:
            # cố gắng trích đoạn {...} lớn nhất rồi sửa
            s_idx, e_idx = L.find("{"), L.rfind("}")
            if s_idx == -1 or e_idx == -1 or e_idx <= s_idx:
                continue
            seg = L[s_idx:e_idx+1]
            for f in ("question","context","answer"):
                seg = _fix_inner_quotes(seg, f)
            seg = re.sub(r",\s*}", "}", seg)
            try:
                yield json.loads(seg)
            except Exception:
                continue

# Gộp & làm sạch hai file vào một JSONL
kept = 0
with open(OUT, "w", encoding="utf-8") as w:
    for path in RAW_FILES:
        with open(path, "rb") as f:
            txt = f.read().decode("utf-8", errors="ignore")
        # bỏ BOM + smart quotes
        bom = codecs.BOM_UTF8.decode("utf-8")
        if txt.startswith(bom): txt = txt.lstrip(bom)
        txt = txt.translate(SMART)

        for obj in _objects_from_text(txt):
            if not all(k in obj for k in ("question","context","answer")):
                continue
            q = str(obj["question"]).strip()
            c = str(obj["context"]).strip()
            a = str(obj["answer"]).strip()
            if not (q and c and a):
                continue
            w.write(json.dumps({"question": q, "context": c, "answer": a}, ensure_ascii=False) + "\n")
            kept += 1

print(f"[OLSM] Wrote {kept} cleaned examples -> {OUT}")

# Load 1 lần từ file sạch và split 80/10/10
olsm_full  = load_dataset("json", data_files=OUT, split="train")
tmp        = olsm_full.train_test_split(test_size=0.2, seed=42)
olsm_train = tmp["train"]
tmp2       = tmp["test"].train_test_split(test_size=0.5, seed=42)
olsm_test, olsm_val = tmp2["train"], tmp2["test"]

olsm_train, olsm_test, olsm_val  # để bạn thấy kích thước ba split


[OLSM] Wrote 22988 cleaned examples -> /kaggle/working/olsm-clean.jsonl


Generating train split: 0 examples [00:00, ? examples/s]

(Dataset({
     features: ['question', 'context', 'answer'],
     num_rows: 18390
 }),
 Dataset({
     features: ['question', 'context', 'answer'],
     num_rows: 2299
 }),
 Dataset({
     features: ['question', 'context', 'answer'],
     num_rows: 2299
 }))

In [ ]:
import shutil

src = "/kaggle/working/olsm-clean.jsonl"
dst = "/kaggle/working/olsm-clean.zip"

# nén file JSONL thành zip
shutil.make_archive(dst.replace(".zip", ""), 'zip', root_dir="/kaggle/working", base_dir="olsm-clean.jsonl")
print("✅ Created zip:", dst)


✅ Created zip: /kaggle/working/olsm-clean.zip


# **XÓA CACHE CŨ**

In [ ]:
import shutil, os

# Thư mục cache cũ
CACHE_DIR = "merged_dataset"

if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR)
    print(f"🗑️ Deleted old cache folder: {CACHE_DIR}")
else:
    print("✅ No old merged_dataset found, will rebuild from scratch")


✅ No old merged_dataset found, will rebuild from scratch


In [ ]:
from datasets import load_dataset, load_from_disk, DatasetDict, interleave_datasets, concatenate_datasets

# Cố gắng nạp dataset đã merge sẵn từ cache "merged_dataset" để tiết kiệm thời gian
try:
    dataset = load_from_disk("merged_dataset")
    print("Loaded Merged Dataset")
except:
    # Nếu không có cache, tiến hành tải và merge các dataset và chia split thủ công

    # =========================
    # 4.0) Local OLSM datasets
    # =========================

    olsm_full = load_dataset("json", data_files="/kaggle/working/olsm-clean.jsonl", split="train")
    _tmp = olsm_full.train_test_split(test_size=0.2, seed=42)
    olsm_train = _tmp["train"]
    _tmp2 = _tmp["test"].train_test_split(test_size=0.5, seed=42)
    olsm_test, olsm_val = _tmp2["train"], _tmp2["test"]
    print("OLSM splits:", olsm_train.num_rows, olsm_val.num_rows, olsm_test.num_rows)
    
        

    # =========================
    # 4.1) b-mc2/sql-create-context
    # =========================
    dataset_scc_train = load_dataset("b-mc2/sql-create-context", split='train[:80%]')       # 80% train
    dataset_scc_test  = load_dataset("b-mc2/sql-create-context", split='train[-20%:-10%]')  # 10% test
    dataset_scc_val   = load_dataset("b-mc2/sql-create-context", split='train[-10%:]')      # 10% val

    # =========================
    # 4.2) Clinton/Text-to-sql-v1 (đổi tên cột)
    # =========================
    dataset_tts_train = load_dataset("Clinton/Text-to-sql-v1", split='train[:80%]')
    dataset_tts_train = dataset_tts_train.remove_columns(['source', 'text'])
    dataset_tts_train = dataset_tts_train.rename_columns({'instruction':'question','input':'context','response':'answer'})

    dataset_tts_test  = load_dataset("Clinton/Text-to-sql-v1", split='train[-20%:-10%]')
    dataset_tts_test  = dataset_tts_test.remove_columns(['source', 'text'])
    dataset_tts_test  = dataset_tts_test.rename_columns({'instruction':'question','input':'context','response':'answer'})

    dataset_tts_val   = load_dataset("Clinton/Text-to-sql-v1", split='train[-10%:]')
    dataset_tts_val   = dataset_tts_val.remove_columns(['source', 'text'])
    dataset_tts_val   = dataset_tts_val.rename_columns({'instruction':'question','input':'context','response':'answer'})

    # =========================
    # 4.3) knowrohit07/know_sql
    # =========================
    dataset_ks_train  = load_dataset("knowrohit07/know_sql", split='validation[:80%]')
    dataset_ks_test   = load_dataset("knowrohit07/know_sql", split='validation[-20%:-10%]')
    dataset_ks_val    = load_dataset("knowrohit07/know_sql", split='validation[-10%:]')


    # 4.4) Ghép 3 nguồn lại bằng interleave_datasets để tăng đa dạng dữ liệu
    # =========================
    from datasets import interleave_datasets, concatenate_datasets, DatasetDict

    # 4.4.1) Interleave 3 bộ HF (mặc định first_exhausted -> dừng ở bộ ngắn nhất để giữ cân bằng)
    mix_train = interleave_datasets([dataset_scc_train, dataset_tts_train, dataset_ks_train])
    mix_test  = interleave_datasets([dataset_scc_test,  dataset_tts_test,  dataset_ks_test])
    mix_val   = interleave_datasets([dataset_scc_val,   dataset_tts_val,   dataset_ks_val])

    # 4.4.2) Gắn nhãn nguồn để thống kê
    def tag(ds, name):
        return ds.map(lambda ex: {"_src": name})

    mix_train, mix_test, mix_val = tag(mix_train, "HF"), tag(mix_test, "HF"), tag(mix_val, "HF")
    olsm_train_t, olsm_test_t, olsm_val_t = tag(olsm_train, "OLSM"), tag(olsm_test, "OLSM"), tag(olsm_val, "OLSM")

    # 4.4.3) Concat với OLSM ĐỨNG TRƯỚC để khi dedup thì OLSM được giữ lại nếu trùng
    dataset = DatasetDict({
        "train":      concatenate_datasets([olsm_train_t, mix_train]).shuffle(seed=42),
        "test":       concatenate_datasets([olsm_test_t,  mix_test]).shuffle(seed=42),
        "validation": concatenate_datasets([olsm_val_t,   mix_val]).shuffle(seed=42),
    })


    # ==== Deduplicate by (question||context||answer) ====
    import hashlib
    from datasets import DatasetDict

    def add_hash(ex):
        s = ex["question"] + "||" + ex["context"] + "||" + ex["answer"]
        ex["_h"] = hashlib.sha1(s.encode("utf-8")).hexdigest()
        return ex

    dataset = DatasetDict({ k: v.map(add_hash) for k, v in dataset.items() })

    def dedup_keep_first(ds):
        seen = set()
        kept_src = {"OLSM": 0, "HF": 0}
        def f(ex):
            h = ex["_h"]
            if h in seen:
                return False
            seen.add(h)
            kept_src[ex.get("_src","HF")] = kept_src.get(ex.get("_src","HF"), 0) + 1
            return True
        out = ds.filter(f)
        # in thống kê nhỏ
        total = out.num_rows
        print(f"Kept total={total} | by source:", kept_src)
        return out

    for split in ["train", "validation", "test"]:
        dataset[split] = dedup_keep_first(dataset[split])
        dataset[split] = dataset[split].remove_columns(["_h","_src"])

    dataset.save_to_disk("merged_dataset")
    print("Merged and Saved Dataset ✅")



dataset  # Hiển thị info


OLSM splits: 18390 2299 2299


README.md: 0.00B [00:00, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/118 [00:00<?, ?B/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/124561 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15001 [00:00<?, ? examples/s]

Merged and Saved Dataset ✅


DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 124561
    })
    test: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 17058
    })
    validation: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 15001
    })
})

In [ ]:
dataset['test'][0:8] # Kiểm tra một mẫu trong tập test

{'question': ['Tell me the 2nd leg for alemannia aachen',
  'Season 9 all the titles were no. in series.',
  'What is Award, when Category is "Best Actor", and when Year is less than 1988?',
  'How large was the crowd when North Melbourne played as the away team?',
  "What's the date when Stu Harris was the Runner-up?",
  'Find enrollments for user ID 102 that happened this year.',
  'What is the Time of the Player with a Rank of 1 or 2 and Notes of FA?',
  'count the number of times patient 15754 was tested for mch in 2104.'],
 'context': ['CREATE TABLE table_56562 (\n    "Team #1" text,\n    "Agg." text,\n    "Team #2" text,\n    "1st leg" text,\n    "2nd leg" text\n)',
  'CREATE TABLE table_3706 (\n    "No. in series" real,\n    "Title" text,\n    "Directed by" text,\n    "Written by" text,\n    "Original air date" text,\n    "Production code" text,\n    "U.S. viewers (million)" text\n)',
  'CREATE TABLE table_name_35 (award VARCHAR, category VARCHAR, year VARCHAR)',
  'CREATE TABLE

# ==============================
# 5) HÀM TIỀN XỬ LÝ (TOKENIZE) + TẠO PROMPT
# ==============================

Ở bước này, cần chuyển đổi các bộ dữ liệu thành dạng hướng dẫn rõ ràng cho mô hình ngôn ngữ lớn (LLM).

Sau đó, tiến hành tiền xử lý dữ liệu prompt-response bằng cách mã hóa (tokenize) để lấy ra các input_ids phục vụ cho quá trình huấn luyện.

Ghi chú: Chuyển dữ liệu dạng (context/question/answer) → (input_ids/labels) cho T5
 Prompt dạng:
 Tables:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n

In [ ]:

from transformers import DataCollatorForSeq2Seq

MAX_INPUT_LEN = 256   # độ dài tối đa của prompt (Tables + Question)
MAX_LABEL_LEN = 128   # độ dài tối đa của Answer/SQL

def tokenize_function(example):
    # Tạo prompt chuẩn dạng:
    # Tables: [schema]
    # Question: [natural language question]
    # Answer: [SQL query]
    start_prompt = "Tables:\n"
    middle_prompt = "\n\nQuestion:\n"
    end_prompt = "\n\nAnswer:\n"

    # Ghép từng cặp context + question thành một prompt hoàn chỉnh
    prompt = [
        start_prompt + context + middle_prompt + question + end_prompt
        for context, question in zip(example["context"], example["question"])
    ]

    # Tokenize đầu vào và đầu ra với padding động + cắt độ dài hợp lý
    model_inputs = tokenizer(
        prompt,
        padding=False,  # 🔻 không ép về max_length
        truncation=True,
        max_length=MAX_INPUT_LEN,
    )

    labels = tokenizer(
        example["answer"],
        padding=False,
        truncation=True,
        max_length=MAX_LABEL_LEN,
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# =========================================================
# 🔹 TẠO DATASET TOKENIZED (CÓ CACHE)
# =========================================================
try:
    tokenized_datasets = load_from_disk("tokenized_datasets")
    print("✅ Loaded tokenized dataset from cache.")
except:
    tokenized_datasets = dataset.map(
        tokenize_function, batched=True, remove_columns=["question", "context", "answer"]
    )
    tokenized_datasets.save_to_disk("tokenized_datasets")
    print("✅ Tokenized dataset saved to cache.")


Map:   0%|          | 0/124561 [00:00<?, ? examples/s]

Map:   0%|          | 0/17058 [00:00<?, ? examples/s]

Map:   0%|          | 0/15001 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/124561 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15001 [00:00<?, ? examples/s]

✅ Tokenized dataset saved to cache.


NameError: name 'finetuned_model' is not defined

In [ ]:
# Hiển thị thông tin tokenized_datasets kiểm tra nhanh khóa/cấu trúc trước khi train
print(tokenized_datasets.keys()) # Hiển thị các split có trong tokenized_datasets
print(tokenized_datasets['train'][0].keys()) # Hiển thị các khóa trong một mẫu của tập train
print(tokenized_datasets['train'][0]['input_ids'][:10]) # Hiển thị 10 token đầu tiên của input_ids
print(tokenized_datasets['train'][0]['labels'][:10]) # Hiển thị 10 token đầu tiên của labels
print(tokenized_datasets) # Hiển thị thông tin dataset

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}") # In kích thước tập train (số mẫu, số cột)
print(f"Validation: {tokenized_datasets['validation'].shape}") # In kích thước tập validation (số mẫu, số cột)
print(f"Test: {tokenized_datasets['test'].shape}") # In kích thước tập test (số mẫu, số cột)

print(tokenized_datasets) # Hiển thị thông tin tokenized_datasets

# ==============================
# 7) KIỂM THỬ ZERO-SHOT VỚI MODEL GỐC (BASELINE)
# ==============================

In [ ]:
import torch # Thư viện PyTorch tăng tốc độ xử lý GPU, tensor

# Định nghĩa prompt & answer mẫu để test nhanh (người dùng cần thay bằng ví dụ thật)
prompt = "Your input prompt here"  # Ví dụ: Tables + Question (cần thay bằng prompt thực)
answer = "Expected human response here"  # Ví dụ: câu SQL chuẩn tương ứng (đáp án chuẩn)

# Đảm bảo model và input cùng trên một thiết bị (CPU hoặc GPU) nếu có
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Di chuyển model sang thiết bị phù hợp (device = CPU hoặc GPU)
original_model.to(device)

# Tokenize đầu vào input và chuyển nó sang cùng thiết bị với model
inputs = tokenizer(prompt, return_tensors='pt').to(device)

# Sinh đầu ra zero-shot từ model gốc (chưa fine-tune)
output = tokenizer.decode( # dùng tokenizer.decode để giải mã token thành text
    original_model.generate( # dùng phương thức generate để sinh text
        inputs["input_ids"], # input_ids của prompt đã tokenized
        max_new_tokens=200, # Giới hạn tối đa 200 token mới sinh
    )[0], # Lấy mảng token đầu tiên trong batch (ở đây batch size=1)
    skip_special_tokens=True # Bỏ qua các token đặc biệt khi giải mã
)

# In kết quả zero-shot baseline
dash_line = '-' * 100 
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}') 
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')


# ==============================
# 8) HUẤN LUYỆN FULL FINE-TUNE (KAGGLE GPU)
# ==============================


**Giải thích:**

- **5e-3**: Đây là giá trị learning rate (tốc độ học) được sử dụng khi huấn luyện mô hình. Giá trị này ảnh hưởng đến tốc độ cập nhật trọng số của mô hình trong quá trình học.
- **Thời gian huấn luyện**: Tổng thời gian để hoàn thành quá trình fine-tune mô hình trên GPU của Kaggle. Nếu dùng máy cá nhân (PC) không có đủ bộ nhớ CUDA thì sẽ không thể huấn luyện với tập dữ liệu lớn.
- **Training Loss**: Độ lỗi (loss) trên tập huấn luyện. Giá trị càng nhỏ chứng tỏ mô hình học tốt trên dữ liệu huấn luyện.
- **Validation Loss**: Độ lỗi trên tập kiểm thử (validation). Giá trị này dùng để đánh giá khả năng tổng quát hóa của mô hình trên dữ liệu chưa từng thấy. Nếu validation loss thấp và gần với training loss, mô hình không bị overfit.

**Kết luận:**  
Mô hình đã được huấn luyện với tốc độ học 5e-3, thời gian gần 3 tiếng trên GPU của Kaggle. Kết quả training loss và validation loss đều thấp, chứng tỏ mô hình học tốt và có khả năng tổng quát hóa tốt trên dữ liệu kiểm thử.

In [ ]:
# Cài thêm để chạy service nếu cần (không bắt buộc)
!pip install fastapi uvicorn transformers torch


In [ ]:
import shutil # Thư viện shutil để nén file zip

# Nén thư mục model đã fine-tune để tiện download từ Kaggle
shutil.make_archive(
    '/kaggle/working/sql_t5_finetuned',  # tên file zip đầu ra
    'zip',                               # định dạng nén
    '/kaggle/working/finetuned_model_2_epoch'  # thư mục model thật sự
)



**Nén toàn bộ dữ liệu quan trọng gồm model, dataset gốc, tokenized dataset, checkpoint training — để tải 1 lần duy nhất.**

In [ ]:
import shutil

# Đường dẫn thư mục làm việc
base_dir = "/kaggle/working"

# Đường dẫn file zip đầu ra
output_zip = f"{base_dir}/olms_sql_full_backup"

# Nén toàn bộ working directory (model + dataset + checkpoint)
shutil.make_archive(output_zip, 'zip', base_dir)

print("Đã nén toàn bộ dữ liệu thành công!")
print(f" File nén nằm tại: {output_zip}.zip")


# 🔁 Resume Training (Strict) — Use Kaggle **folder** dataset (no zip)
> **Lưu ý:** Phần dưới **chỉ thêm cell mới để resume**, **không** thay đổi bất kỳ tham số huấn luyện cũ nào.
> `TrainingArguments` **được giữ nguyên** (dùng lại biến `args` đã khai báo ở trên).


In [ ]:
# === [CHANGE] Chỉ định dataset chứa checkpoint (model v3) và merged_dataset mới (ở WORKING) ===
DATASET_MODEL = "/kaggle/input/sql-t5-finetuned-v3"   # ✅ model v3: files nằm ở ROOT

import os, shutil

# === [CHANGE] Tìm checkpoint trong dataset model v3 (files ở root)
if os.path.exists(os.path.join(DATASET_MODEL, "config.json")) and \
   (os.path.exists(os.path.join(DATASET_MODEL, "pytorch_model.bin")) or 
    os.path.exists(os.path.join(DATASET_MODEL, "model.safetensors"))):
    CKPT_SRC = DATASET_MODEL                     # ✅ lấy trực tiếp root làm checkpoint src
else:
    raise FileNotFoundError("Không thấy checkpoint (config.json + weights) trong sql-t5-finetuned-v3.")

# === Copy checkpoint sang vùng ghi được để resume (KHÔNG đổi tên biến đích) ===
WORK      = "/kaggle/working"
CKPT_WORK = os.path.join(WORK, "sql_t5_finetuned")
os.makedirs(CKPT_WORK, exist_ok=True)
shutil.copytree(CKPT_SRC, CKPT_WORK, dirs_exist_ok=True)
print("Checkpoint copied to:", CKPT_WORK)

# === Ưu tiên dùng merged_dataset vừa merge ở WORKING ===
# Nếu chưa có, nhưng có file jsonl mới (olsm-clean.jsonl) thì tự build merged_dataset tối thiểu rồi dùng.
TOKENIZED_INPUT = os.path.join(DATASET_MODEL, "tokenized_datasets")  # bộ tokenized cũ (KHÔNG dùng)
RAW_DIR         = "merged_dataset"                                    # mong muốn: save_to_disk ở WORKING
JSONL_FALLBACK  = "/kaggle/working/olsm-clean.jsonl"                  # nếu chưa có merged_dataset

USE_OLD_TOKENIZED = False
HAS_TOKENIZED   = os.path.exists(TOKENIZED_INPUT) and USE_OLD_TOKENIZED
HAS_RAW         = os.path.exists(RAW_DIR)

print("HAS_TOKENIZED =", HAS_TOKENIZED, "| HAS_RAW(merged_dataset) =", HAS_RAW)

# === [CHANGE NHẸ] Fallback: nếu chưa có merged_dataset nhưng có jsonl, convert nhanh thành merged_dataset ===
if (not HAS_RAW) and os.path.exists(JSONL_FALLBACK):
    from datasets import load_dataset
    print("⚠️  merged_dataset chưa có. Đang build tạm từ:", JSONL_FALLBACK)
    ds_tmp = load_dataset("json", data_files=JSONL_FALLBACK, split="train")
    # đảm bảo 3 cột chuẩn
    need_cols = {"question","context","answer"}
    assert need_cols.issubset(ds_tmp.column_names), f"JSONL thiếu cột. Có: {ds_tmp.column_names}"
    # chia 80/10/10 và save_to_disk để các cell sau load_from_disk như cũ
    _spl = ds_tmp.train_test_split(test_size=0.2, seed=42)
    _train = _spl["train"]
    _tmp2  = _spl["test"].train_test_split(test_size=0.5, seed=42)
    _test, _val = _tmp2["train"], _tmp2["test"]
    from datasets import DatasetDict
    ds_built = DatasetDict({"train": _train, "test": _test, "validation": _val})
    ds_built.save_to_disk(RAW_DIR)
    HAS_RAW = True
    print("✅ Đã build và lưu merged_dataset/ (tối thiểu) từ jsonl.")


Checkpoint copied to: /kaggle/working/sql_t5_finetuned
HAS_TOKENIZED = False | HAS_RAW(merged_dataset) = True


In [ ]:

# === [NO CHANGE TO LOGIC] Load tokenizer & model từ checkpoint cũ ===
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained(CKPT_WORK)
model     = T5ForConditionalGeneration.from_pretrained(CKPT_WORK)
model.config.use_cache = False  # giữ nguyên an toàn cho Trainer
print("Loaded model/tokenizer from:", CKPT_WORK)


Loaded model/tokenizer from: /kaggle/working/sql_t5_finetuned


In [ ]:
# === Chuẩn bị dữ liệu: dùng merged_dataset mới ở WORKING ===
from datasets import load_from_disk
import os

if HAS_TOKENIZED:
    # (Không dùng vì USE_OLD_TOKENIZED=False)
    ds = load_from_disk(TOKENIZED_INPUT)
    train_ds = ds["train"]
    eval_ds  = ds.get("validation", None) or ds.get("eval", None)
    print("Loaded tokenized_datasets from:", TOKENIZED_INPUT)

elif HAS_RAW:
    # merged_dataset là DatasetDict đã save_to_disk
    ds = load_from_disk(RAW_DIR)
    raw_train = ds["train"]
    raw_val   = ds["validation"]

    # Tên cột chuẩn
    INPUT_Q  = "question"
    INPUT_C  = "context"
    TARGET_A = "answer"

    # [CHANGE] Dùng đúng prompt đang infer: "Tables:\n{context}\n\nQuestion:\n{q}\n\nAnswer:\n"
    def to_prompt(q, ctx):
        ctx = (ctx or "").strip()
        if ctx:
            return f"Tables:\n{ctx}\n\nQuestion:\n{q}\n\nAnswer:\n"
        # fallback nếu context trống
        return f"translate to sql: {q}\nAnswer:\n"

    # [CHANGE] Bỏ as_target_tokenizer (API cũ). Dùng text_target cho nhãn.
    def preprocess(examples):
        inputs  = [to_prompt(q, c) for q, c in zip(examples[INPUT_Q], examples[INPUT_C])]
        targets = examples[TARGET_A]
        model_inputs = tokenizer(inputs, max_length=256, truncation=True)  # padding do collator xử lý
        labels = tokenizer(text_target=targets, max_length=256, truncation=True)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    train_ds = raw_train.map(preprocess, batched=True, remove_columns=raw_train.column_names)
    eval_ds  = raw_val.map(preprocess,   batched=True, remove_columns=raw_val.column_names)
    print("Tokenized from merged_dataset:", RAW_DIR)

    # lưu để lần sau load nhanh
    from datasets import DatasetDict
    tok_out = "/kaggle/working/tokenized_datasets_v4"
    DatasetDict({"train": train_ds, "validation": eval_ds}).save_to_disk(tok_out)
    print("Saved new tokenized dataset to:", tok_out)

else:
    raise FileNotFoundError("Không tìm thấy merged_dataset mới ở WORKING.")

print("Sizes:", len(train_ds), len(eval_ds) if eval_ds is not None else 0)


Map:   0%|          | 0/124561 [00:00<?, ? examples/s]

Map:   0%|          | 0/15001 [00:00<?, ? examples/s]

Tokenized from merged_dataset: merged_dataset


Saving the dataset (0/1 shards):   0%|          | 0/124561 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15001 [00:00<?, ? examples/s]

Saved new tokenized dataset to: /kaggle/working/tokenized_datasets_v4
Sizes: 124561 15001


Tự tạo TrainingArguments mới (nhưng phải dùng đúng biến)

In [ ]:
import os
print(os.listdir(CKPT_WORK)[:20])

['added_tokens.json', 'special_tokens_map.json', 'config.json', 'generation_config.json', 'model.safetensors', 'spiece.model', 'tokenizer_config.json']


In [ ]:
import math, os, json, time, torch

# 1) Kiểm tra đường dẫn & loại dữ liệu đang dùng
using_tokenized = 'train_ds' in globals() and hasattr(train_ds, '__len__')
src = "train_ds (tokenized_datasets)" if using_tokenized else "tokenized_datasets['train']" if 'tokenized_datasets' in globals() else "UNKNOWN"

if using_tokenized:
    N_train = len(train_ds)
    N_eval  = len(eval_ds) if 'eval_ds' in globals() and eval_ds is not None else 0
else:
    assert 'tokenized_datasets' in globals(), "Không tìm thấy biến train_ds hoặc tokenized_datasets."
    N_train = len(tokenized_datasets["train"])
    N_eval  = len(tokenized_datasets["validation"]) if "validation" in tokenized_datasets else 0

print(f"🧾 Using dataset source: {src}")
print(f"🔢 Train size: {N_train} | Eval size: {N_eval}")

# 2) Tính steps/epoch theo args hiện tại
bs = 4   # per_device_train_batch_size (bạn đang để 4)
ga = 4   # gradient_accumulation_steps (bạn đang để 4)
world = 1  # Kaggle T4 = 1 GPU
steps_per_epoch = math.ceil( math.ceil(N_train / (bs*world)) / ga )
print(f"📈 Steps/epoch (theory): {steps_per_epoch} with effective batch = {bs*ga*world}")

# 3) Kiểm tra model & checkpoint hiện tại
print("📦 CKPT_WORK:", CKPT_WORK if 'CKPT_WORK' in globals() else "(not set)")
if 'CKPT_WORK' in globals() and os.path.exists(CKPT_WORK):
    print("CKPT files (head):", os.listdir(CKPT_WORK)[:12])

# 4) Check gradient checkpointing on/off
if hasattr(model, "is_gradient_checkpointing"):
    print("⚙️ gradient_checkpointing:", model.is_gradient_checkpointing)
else:
    print("⚙️ gradient_checkpointing: unknown flag; assuming enabled if you called gradient_checkpointing_enable()")

# 5) In thử 2 mẫu đầu để chắc chắn đúng tập
if using_tokenized:
    try:
        print("👀 Sample[0] keys:", train_ds[0].keys())
    except Exception as e:
        print("Sample preview error:", e)
else:
    try:
        print("👀 Sample[0] keys:", tokenized_datasets["train"][0].keys())
    except Exception as e:
        print("Sample preview error:", e)


🧾 Using dataset source: train_ds (tokenized_datasets)
🔢 Train size: 124561 | Eval size: 15001
📈 Steps/epoch (theory): 7786 with effective batch = 16
📦 CKPT_WORK: /kaggle/working/sql_t5_finetuned
CKPT files (head): ['added_tokens.json', 'special_tokens_map.json', 'config.json', 'generation_config.json', 'model.safetensors', 'spiece.model', 'tokenizer_config.json']
⚙️ gradient_checkpointing: False
👀 Sample[0] keys: dict_keys(['input_ids', 'attention_mask', 'labels'])


In [ ]:
# Chạy cell này ngay trước khi tạo Trainer (KHÔNG đổi args)
model.gradient_checkpointing_disable()   # <- tắt để tăng tốc
model.config.use_cache = False           # giữ như cũ để không bơm KV-cache
torch.cuda.empty_cache()


In [ ]:
import os, torch

# Bật TF32 trên GPU (PyTorch 2.x, T4/AMPERE hỗ trợ) – tăng tốc matmul
torch.backends.cuda.matmul.allow_tf32 = True
try:
    torch.set_float32_matmul_precision("high")  # nếu PyTorch hỗ trợ
except Exception:
    pass

# Cho phép tokenizer xử lý song song (giảm overhead CPU)
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# Nếu kernel bị nghẽn CPU, giới hạn threads để ổn định (tuỳ máy)
try:
    torch.set_num_threads(2)
except Exception:
    pass

# Nếu VRAM đủ: tắt gradient checkpointing để tăng tốc
try:
    model.gradient_checkpointing_disable()
except Exception:
    pass

torch.cuda.empty_cache()


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "google-t5/t5-small"  # fallback base
finetuned_model = T5ForConditionalGeneration.from_pretrained(CKPT_WORK).to(device)
tokenizer = T5Tokenizer.from_pretrained(CKPT_WORK)
to_train = True  # muốn tiếp tục train thêm

# **KIỂM TRA TRƯỚC KHI TRAIN**

In [ ]:
import os, torch, math

print("🔍 ====== PRE-TRAIN SANITY CHECK ======")

# 1️⃣ Kiểm tra thiết bị và model checkpoint
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🧠 Device: {device.upper()} | GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU only'}")

if 'CKPT_WORK' in globals() and os.path.exists(CKPT_WORK):
    print(f"📦 Checkpoint in use: {CKPT_WORK}")
    ckpt_files = os.listdir(CKPT_WORK)[:10]
    print("📁 Sample files:", ckpt_files)
else:
    print("⚠️ CKPT_WORK not found — model may be base or loaded directly.")

# 2️⃣ Kiểm tra model & tokenizer
try:
    _ = finetuned_model.state_dict()
    print("✅ Model loaded:", finetuned_model.__class__.__name__)
except Exception as e:
    raise RuntimeError(f"❌ Model not loaded properly: {e}")

try:
    print("✅ Tokenizer vocab size:", len(tokenizer))
except Exception as e:
    raise RuntimeError(f"❌ Tokenizer not found: {e}")

# 3️⃣ Kiểm tra dataset
if 'train_ds' in globals():
    ds_train = train_ds
    ds_eval  = eval_ds if 'eval_ds' in globals() else None
    print("📊 Dataset source: 'train_ds' (custom tokenized)")
elif 'tokenized_datasets' in globals():
    ds_train = tokenized_datasets["train"]
    ds_eval  = tokenized_datasets.get("validation", None)
    print("📊 Dataset source: 'tokenized_datasets'")
else:
    raise RuntimeError("❌ No dataset found! You must load tokenized dataset first.")

print(f"   → Train samples: {len(ds_train):,}")
if ds_eval is not None:
    print(f"   → Eval samples:  {len(ds_eval):,}")
else:
    print("   → Eval samples:  (None — skipped eval)")

# 4️⃣ Kiểm tra 1 sample ngẫu nhiên
try:
    ex = ds_train[0]
    print("👀 Example keys:", list(ex.keys()))
    if "input_ids" in ex and "labels" in ex:
        print("✅ Tokenized format confirmed (input_ids + labels).")
    else:
        print("⚠️ Non-tokenized format — bạn có thể cần map(preprocess) lại.")
except Exception as e:
    print("⚠️ Could not preview sample:", e)

# 5️⃣ Tính toán số bước mỗi epoch
bs = 4
ga = 4
world = 1
steps_per_epoch = math.ceil(len(ds_train) / (bs * ga * world))
print(f"📈 Estimated steps/epoch: {steps_per_epoch:,} (effective batch = {bs*ga})")

print("🔍 Sanity check OK ✅ — Ready to train if everything looks correct.")


🔍 ====== PRE-TRAIN SANITY CHECK ======
🧠 Device: CUDA | GPU: Tesla T4
📦 Checkpoint in use: /kaggle/working/sql_t5_finetuned
📁 Sample files: ['added_tokens.json', 'special_tokens_map.json', 'config.json', 'generation_config.json', 'model.safetensors', 'spiece.model', 'tokenizer_config.json']
✅ Model loaded: T5ForConditionalGeneration
✅ Tokenizer vocab size: 32100
📊 Dataset source: 'train_ds' (custom tokenized)
   → Train samples: 124,561
   → Eval samples:  15,001
👀 Example keys: ['input_ids', 'attention_mask', 'labels']
✅ Tokenized format confirmed (input_ids + labels).
📈 Estimated steps/epoch: 7,786 (effective batch = 16)
🔍 Sanity check OK ✅ — Ready to train if everything looks correct.


In [ ]:
import math, torch

print("==== Trainer runtime args check ====")
print("per_device_train_batch_size:", trainer.args.per_device_train_batch_size)
print("gradient_accumulation_steps:", trainer.args.gradient_accumulation_steps)
print("world_size:", trainer.args.world_size)

dl = trainer.get_train_dataloader()
batch = next(iter(dl))
print("len(train_ds):", len(train_data_for_trainer))
print("len(train_dataloader) (num mini-batches):", len(dl))
print("actual batch size from tensor:", batch["input_ids"].shape[0])
print("=> expected optimizer steps (ceil):",
      math.ceil(len(dl)/trainer.args.gradient_accumulation_steps))


==== Trainer runtime args check ====
per_device_train_batch_size: 4
gradient_accumulation_steps: 1
world_size: 1
len(train_ds): 124561
len(train_dataloader) (num mini-batches): 15571
actual batch size from tensor: 8
=> expected optimizer steps (ceil): 15571


In [ ]:
# ====== RESET TRAINER STATE (quan trọng) ======
import torch, gc
for v in ["trainer", "args", "training_args"]:
    if v in globals():
        del globals()[v]
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# ====== SAFETY: đảm bảo dataset đã tokenized ======
assert 'train_ds' in globals() or 'tokenized_datasets' in globals(), "❌ Chưa có tokenized dataset."

train_data_for_trainer = train_ds if 'train_ds' in globals() else tokenized_datasets["train"]
eval_data_for_trainer  = (eval_ds if 'eval_ds' in globals()
                          else tokenized_datasets.get("validation", None))

In [ ]:
# ====== CẤU HÌNH HỌC SÂU (deep) ======
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

output_dir = f"./sql-training-deep"

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=3e-3,                 # deep hơn -> hạ LR nhẹ cho ổn định
    num_train_epochs=3,                 # học kỹ hơn
    per_device_train_batch_size=4,      # batch nhỏ giúp học tinh hơn
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,      # gộp ít hơn -> nhiều optimizer steps hơn
    weight_decay=0.01,
    logging_steps=50,
    eval_strategy="no",           # giữ nhẹ
    save_strategy="steps",              #  lưu theo bước
    save_steps=1000,                    #  mỗi 1000 bước lưu 1 ckpt
    fp16=True,                          # tiết kiệm VRAM
    optim="adafactor",                  # tiết kiệm bộ nhớ
    report_to="none",
    group_by_length=False,              # TRÁNH sampler tự gom độ dài làm bạn khó đọc batch/steps
    dataloader_drop_last=False,
    dataloader_num_workers=0,           # 0 nếu lỗi môi trường
    dataloader_pin_memory=False,
    seed=42,
    save_total_limit=2,          # chỉ giữ lại 2 checkpoint gần nhất
    warmup_ratio=0.03,           # ~3% bước đầu giảm rủi ro “giật” LR
    lr_scheduler_type="linear",  # lịch trình LR đơn giản, ổn định
)

# Data collator cho seq2seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=finetuned_model)


In [ ]:
# ====== KHỞI TẠO TRAINER ======
trainer = Trainer(
    model=finetuned_model,
    args=training_args,
    train_dataset=train_data_for_trainer,
    # eval_dataset không truyền vì evaluation_strategy="no"
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
# ====== SANITY CHECK TRƯỚC KHI TRAIN ======
import math
dl = trainer.get_train_dataloader()
first = next(iter(dl))
print("==== Runtime sanity check ====")
print("Train size:", len(train_data_for_trainer))
print("per_device_train_batch_size:", trainer.args.per_device_train_batch_size)
print("gradient_accumulation_steps:", trainer.args.gradient_accumulation_steps)
print("len(train_dataloader):", len(dl))
print("actual batch size (tensor):", first["input_ids"].shape[0])
steps_per_epoch = math.ceil(len(dl) / trainer.args.gradient_accumulation_steps)
print("=> expected optimizer steps/epoch:", steps_per_epoch)

==== Runtime sanity check ====
Train size: 124561
per_device_train_batch_size: 4
gradient_accumulation_steps: 1
len(train_dataloader): 15571
actual batch size (tensor): 8
=> expected optimizer steps/epoch: 15571


# TRAIN 3EP - 2.5h

In [ ]:
from pathlib import Path
from transformers.trainer_callback import TrainerCallback

# ====== Heartbeat callback (in nhịp tim mỗi 500 bước) ======
class Heartbeat(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 500 == 0:
            print(f"[HB] step={state.global_step}/{state.max_steps} | epoch={state.epoch:.2f}")

trainer.add_callback(Heartbeat())   # ✅ gắn callback vào trainer

# ====== Resume từ checkpoint gần nhất ======
ckpts = sorted(Path(output_dir).glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1]))
resume_ckpt = str(ckpts[-1]) if ckpts else None

import torch
torch.cuda.empty_cache()

if resume_ckpt:
    print("Resuming from:", resume_ckpt)
    trainer.train(resume_from_checkpoint=resume_ckpt)
else:
    print("No checkpoint found. Starting fresh.")
    trainer.train()

# ====== SAVE ======
finetuned_model.save_pretrained("finetuned_model_3_epoch")
tokenizer.save_pretrained("finetuned_model_3_epoch")
print("✅ Saved to: finetuned_model_3_epoch")


No checkpoint found. Starting fresh.


Step,Training Loss
50,0.181900
100,0.134100
150,0.159200
200,0.182500
250,0.149100
300,0.117200
350,0.121300
400,0.168000
450,0.161300
500,0.166700


[HB] step=500/46713 | epoch=0.03
[HB] step=1000/46713 | epoch=0.06
[HB] step=1500/46713 | epoch=0.10
[HB] step=2000/46713 | epoch=0.13
[HB] step=2500/46713 | epoch=0.16
[HB] step=3000/46713 | epoch=0.19
[HB] step=3500/46713 | epoch=0.22
[HB] step=4000/46713 | epoch=0.26
[HB] step=4500/46713 | epoch=0.29
[HB] step=5000/46713 | epoch=0.32
[HB] step=5500/46713 | epoch=0.35
[HB] step=6000/46713 | epoch=0.39
[HB] step=6500/46713 | epoch=0.42
[HB] step=7000/46713 | epoch=0.45
[HB] step=7500/46713 | epoch=0.48
[HB] step=8000/46713 | epoch=0.51
[HB] step=8500/46713 | epoch=0.55
[HB] step=9000/46713 | epoch=0.58
[HB] step=9500/46713 | epoch=0.61
[HB] step=10000/46713 | epoch=0.64
[HB] step=10500/46713 | epoch=0.67
[HB] step=11000/46713 | epoch=0.71
[HB] step=11500/46713 | epoch=0.74
[HB] step=12000/46713 | epoch=0.77
[HB] step=12500/46713 | epoch=0.80
[HB] step=13000/46713 | epoch=0.83
[HB] step=13500/46713 | epoch=0.87
[HB] step=14000/46713 | epoch=0.90
[HB] step=14500/46713 | epoch=0.93
[HB] 

# TRAIN 1EP - 3.5h

In [ ]:
# ===== Cố gắng nạp model đã fine-tune nếu có =====
try:
    # [NOTE] ĐỔI TÊN THƯ MỤC: dùng epoch 3 hiện có trong /kaggle/working
    finetuned_model = T5ForConditionalGeneration.from_pretrained(
        "/kaggle/working/finetuned_model_3_epoch"   # [NOTE]
    ).to(device)
    tokenizer = T5Tokenizer.from_pretrained(
        "/kaggle/working/finetuned_model_3_epoch"   # [NOTE]
    )

    to_train = True   # [NOTE] để tiếp tục huấn luyện thêm 1 epoch
except Exception as e:
    to_train = True
    if device == "cuda":
        finetuned_model = T5ForConditionalGeneration.from_pretrained(
            base_source, torch_dtype=torch.bfloat16
        ).to(device)
    else:
        finetuned_model = T5ForConditionalGeneration.from_pretrained(base_source).to(device)
    tokenizer = T5Tokenizer.from_pretrained(base_source)


In [ ]:
import os, time, torch
from transformers import TrainingArguments, Trainer

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
finetuned_model.config.use_cache = False
finetuned_model.gradient_checkpointing_enable()  # như cũ

if to_train:
    output_dir = f"./sql-training-{int(time.time())}"

    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=5e-3,
        num_train_epochs=1,                 # [NOTE] chỉ thêm đúng 1 epoch
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,
        weight_decay=0.01,
        logging_steps=50,
        eval_strategy="no",
        save_strategy="epoch",
        fp16=True,
        optim="adafactor",
        report_to="none",
    )

    trainer = Trainer(
        model=finetuned_model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_datasets["train"],
    )

    # (Tuỳ chọn) sanity check trước khi train
    import math
    dl = trainer.get_train_dataloader()
    print("==== Runtime sanity check ====")
    print("Train size:", len(tokenized_datasets["train"]))
    print("per_device_train_batch_size:", training_args.per_device_train_batch_size)
    print("gradient_accumulation_steps:", training_args.gradient_accumulation_steps)
    print("len(train_dataloader):", len(dl))
    steps_per_epoch = math.ceil(len(dl) / training_args.gradient_accumulation_steps)
    print("=> expected optimizer steps/epoch:", steps_per_epoch)

    torch.cuda.empty_cache()
    # [NOTE] KHÔNG truyền resume_from_checkpoint — vì đã load trọng số epoch-3 vào model rồi
    trainer.train()
    
print("✅ Training completed successfully!")
finetuned_model.save_pretrained("finetuned_model_4_epoch")   # [NOTE] đổi tên thư mục lưu
tokenizer.save_pretrained("finetuned_model_4_epoch")

==== Runtime sanity check ====
Train size: 124561
per_device_train_batch_size: 4
gradient_accumulation_steps: 4
len(train_dataloader): 15571
=> expected optimizer steps/epoch: 3893


Step,Training Loss
50,0.161000
100,0.173900
150,0.195800
200,0.210600
250,0.196000
300,0.215600
350,0.202800
400,0.199400
450,0.209500
500,0.205100


✅ Training completed successfully!


('finetuned_model_4_epoch/tokenizer_config.json',
 'finetuned_model_4_epoch/special_tokens_map.json',
 'finetuned_model_4_epoch/spiece.model',
 'finetuned_model_4_epoch/added_tokens.json')

In [ ]:
import shutil, os

#  Thư mục model cần nén
model_dir = "/kaggle/working/finetuned_model_4_epoch"

#  Đường dẫn file zip đầu ra (không thêm .zip ở đây)
output_zip = f"/kaggle/working/finetuned_model_４_epoch_backup"

#  Tiến hành nén riêng thư mục model
print("🔄 Đang nén model:", model_dir)
shutil.make_archive(output_zip, 'zip', model_dir)

#  Kiểm tra kết quả
zip_path = f"{output_zip}.zip"
if os.path.exists(zip_path):
    size_mb = os.path.getsize(zip_path) / (1024**2)
    print("✅ Đã nén model thành công!")
    print(f"📦 File nén nằm tại: {zip_path}")
    print(f"💾 Dung lượng: {size_mb:.2f} MB")
else:
    print("⚠️ Có lỗi xảy ra: Không tìm thấy file nén sau khi tạo.")


🔄 Đang nén model: /kaggle/working/finetuned_model_4_epoch
✅ Đã nén model thành công!
📦 File nén nằm tại: /kaggle/working/finetuned_model_４_epoch_backup.zip
💾 Dung lượng: 214.56 MB
